In [ ]:
!pip -vvv install prettytable

In [ ]:
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import json
from sklearn.metrics import roc_curve


def initialize_model(num_classes, feature_extract, use_pretrained=True):

    model_ft = models.inception_v3(pretrained=use_pretrained)
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299
    
    return model_ft, input_size






criterion = nn.CrossEntropyLoss()

num_classes = 196
batch_size = 16
num_epochs = 75
feature_extract = False

model_ft, input_size = initialize_model(num_classes, feature_extract, use_pretrained=True)
params_to_update = model_ft.parameters()
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


first_parameter = next(model_ft.parameters())
input_shape = first_parameter.size()
print(input_shape)



from prettytable import PrettyTable


def count_parameters(model):
    table = PrettyTable(['Modules', 'Parameters'])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f'Total Trainable Params: {total_params}')
    return total_params


params = count_parameters(model_ft)
print(params)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
torch.Size([32, 3, 3, 3])
+--------------------------------------+------------+
|               Modules                | Parameters |
+--------------------------------------+------------+
|      Conv2d_1a_3x3.conv.weight       |    864     |
|       Conv2d_1a_3x3.bn.weight        |     32     |
|        Conv2d_1a_3x3.bn.bias         |     32     |
|      Conv2d_2a_3x3.conv.weight       |    9216    |
|       Conv2d_2a_3x3.bn.weight        |     32     |
|        Conv2d_2a_3x3.bn.bias         |     32     |
|      Conv2d_2b_3x3.conv.weight       |   18432    |
|       Conv2d_2b_3x3.bn.weight        |     64     |
|        Conv2d_2b_3x3.bn.bias         |     64     |
|      Conv2d_3b_1x1.conv.weight       |    5120    |
|       Conv2d_3b_1x1.bn.weight        |     80     |
|        Conv2d_3b_1x1.bn.bias         |     80     |
|      Conv2d_4a_3x3.conv.weight       |   138240   |
|       Conv2d_4a_3x3.bn.weight        |    192     |
|        Conv2d_4a_3x3.bn.bias         |    192     |
|    Mixed_5b.branch1x1.conv.weight    |   12288    |
|     Mixed_5b.branch1x1.bn.weight     |     64     |
|      Mixed_5b.branch1x1.bn.bias      |     64     |
|   Mixed_5b.branch5x5_1.conv.weight   |    9216    |
|    Mixed_5b.branch5x5_1.bn.weight    |     48     |
|     Mixed_5b.branch5x5_1.bn.bias     |     48     |
|   Mixed_5b.branch5x5_2.conv.weight   |   76800    |
|    Mixed_5b.branch5x5_2.bn.weight    |     64     |
|     Mixed_5b.branch5x5_2.bn.bias     |     64     |
| Mixed_5b.branch3x3dbl_1.conv.weight  |   12288    |
|  Mixed_5b.branch3x3dbl_1.bn.weight   |     64     |
|   Mixed_5b.branch3x3dbl_1.bn.bias    |     64     |
| Mixed_5b.branch3x3dbl_2.conv.weight  |   55296    |
|  Mixed_5b.branch3x3dbl_2.bn.weight   |     96     |
|   Mixed_5b.branch3x3dbl_2.bn.bias    |     96     |
| Mixed_5b.branch3x3dbl_3.conv.weight  |   82944    |
|  Mixed_5b.branch3x3dbl_3.bn.weight   |     96     |
|   Mixed_5b.branch3x3dbl_3.bn.bias    |     96     |
|   Mixed_5b.branch_pool.conv.weight   |    6144    |
|    Mixed_5b.branch_pool.bn.weight    |     32     |
|     Mixed_5b.branch_pool.bn.bias     |     32     |
|    Mixed_5c.branch1x1.conv.weight    |   16384    |
|     Mixed_5c.branch1x1.bn.weight     |     64     |
|      Mixed_5c.branch1x1.bn.bias      |     64     |
|   Mixed_5c.branch5x5_1.conv.weight   |   12288    |
|    Mixed_5c.branch5x5_1.bn.weight    |     48     |
|     Mixed_5c.branch5x5_1.bn.bias     |     48     |
|   Mixed_5c.branch5x5_2.conv.weight   |   76800    |
|    Mixed_5c.branch5x5_2.bn.weight    |     64     |
|     Mixed_5c.branch5x5_2.bn.bias     |     64     |
| Mixed_5c.branch3x3dbl_1.conv.weight  |   16384    |
|  Mixed_5c.branch3x3dbl_1.bn.weight   |     64     |
|   Mixed_5c.branch3x3dbl_1.bn.bias    |     64     |
| Mixed_5c.branch3x3dbl_2.conv.weight  |   55296    |
|  Mixed_5c.branch3x3dbl_2.bn.weight   |     96     |
|   Mixed_5c.branch3x3dbl_2.bn.bias    |     96     |
| Mixed_5c.branch3x3dbl_3.conv.weight  |   82944    |
|  Mixed_5c.branch3x3dbl_3.bn.weight   |     96     |
|   Mixed_5c.branch3x3dbl_3.bn.bias    |     96     |
|   Mixed_5c.branch_pool.conv.weight   |   16384    |
|    Mixed_5c.branch_pool.bn.weight    |     64     |
|     Mixed_5c.branch_pool.bn.bias     |     64     |
|    Mixed_5d.branch1x1.conv.weight    |   18432    |
|     Mixed_5d.branch1x1.bn.weight     |     64     |
|      Mixed_5d.branch1x1.bn.bias      |     64     |
|   Mixed_5d.branch5x5_1.conv.weight   |   13824    |
|    Mixed_5d.branch5x5_1.bn.weight    |     48     |
|     Mixed_5d.branch5x5_1.bn.bias     |     48     |
|   Mixed_5d.branch5x5_2.conv.weight   |   76800    |
|    Mixed_5d.branch5x5_2.bn.weight    |     64     |
|     Mixed_5d.branch5x5_2.bn.bias     |     64     |
| Mixed_5d.branch3x3dbl_1.conv.weight  |   18432    |
|  Mixed_5d.branch3x3dbl_1.bn.weight   |     64     |
|   Mixed_5d.branch3x3dbl_1.bn.bias    |     64     |
| Mixed_5d.branch3x3dbl_2.conv.weight  |   55296    |
|  Mixed_5d.branch3x3dbl_2.bn.weight   |     96     |
|   Mixed_5d.branch3x3dbl_2.bn.bias    |     96     |
| Mixed_5d.branch3x3dbl_3.conv.weight  |   82944    |
|  Mixed_5d.branch3x3dbl_3.bn.weight   |     96     |
|   Mixed_5d.branch3x3dbl_3.bn.bias    |     96     |
|   Mixed_5d.branch_pool.conv.weight   |   18432    |
|    Mixed_5d.branch_pool.bn.weight    |     64     |
|     Mixed_5d.branch_pool.bn.bias     |     64     |
|    Mixed_6a.branch3x3.conv.weight    |   995328   |
|     Mixed_6a.branch3x3.bn.weight     |    384     |
|      Mixed_6a.branch3x3.bn.bias      |    384     |
| Mixed_6a.branch3x3dbl_1.conv.weight  |   18432    |
|  Mixed_6a.branch3x3dbl_1.bn.weight   |     64     |
|   Mixed_6a.branch3x3dbl_1.bn.bias    |     64     |
| Mixed_6a.branch3x3dbl_2.conv.weight  |   55296    |
|  Mixed_6a.branch3x3dbl_2.bn.weight   |     96     |
|   Mixed_6a.branch3x3dbl_2.bn.bias    |     96     |
| Mixed_6a.branch3x3dbl_3.conv.weight  |   82944    |
|  Mixed_6a.branch3x3dbl_3.bn.weight   |     96     |
|   Mixed_6a.branch3x3dbl_3.bn.bias    |     96     |
|    Mixed_6b.branch1x1.conv.weight    |   147456   |
|     Mixed_6b.branch1x1.bn.weight     |    192     |
|      Mixed_6b.branch1x1.bn.bias      |    192     |
|   Mixed_6b.branch7x7_1.conv.weight   |   98304    |
|    Mixed_6b.branch7x7_1.bn.weight    |    128     |
|     Mixed_6b.branch7x7_1.bn.bias     |    128     |
|   Mixed_6b.branch7x7_2.conv.weight   |   114688   |
|    Mixed_6b.branch7x7_2.bn.weight    |    128     |
|     Mixed_6b.branch7x7_2.bn.bias     |    128     |
|   Mixed_6b.branch7x7_3.conv.weight   |   172032   |
|    Mixed_6b.branch7x7_3.bn.weight    |    192     |
|     Mixed_6b.branch7x7_3.bn.bias     |    192     |
| Mixed_6b.branch7x7dbl_1.conv.weight  |   98304    |
|  Mixed_6b.branch7x7dbl_1.bn.weight   |    128     |
|   Mixed_6b.branch7x7dbl_1.bn.bias    |    128     |
| Mixed_6b.branch7x7dbl_2.conv.weight  |   114688   |
|  Mixed_6b.branch7x7dbl_2.bn.weight   |    128     |
|   Mixed_6b.branch7x7dbl_2.bn.bias    |    128     |
| Mixed_6b.branch7x7dbl_3.conv.weight  |   114688   |
|  Mixed_6b.branch7x7dbl_3.bn.weight   |    128     |
|   Mixed_6b.branch7x7dbl_3.bn.bias    |    128     |
| Mixed_6b.branch7x7dbl_4.conv.weight  |   114688   |
|  Mixed_6b.branch7x7dbl_4.bn.weight   |    128     |
|   Mixed_6b.branch7x7dbl_4.bn.bias    |    128     |
| Mixed_6b.branch7x7dbl_5.conv.weight  |   172032   |
|  Mixed_6b.branch7x7dbl_5.bn.weight   |    192     |
|   Mixed_6b.branch7x7dbl_5.bn.bias    |    192     |
|   Mixed_6b.branch_pool.conv.weight   |   147456   |
|    Mixed_6b.branch_pool.bn.weight    |    192     |
|     Mixed_6b.branch_pool.bn.bias     |    192     |
|    Mixed_6c.branch1x1.conv.weight    |   147456   |
|     Mixed_6c.branch1x1.bn.weight     |    192     |
|      Mixed_6c.branch1x1.bn.bias      |    192     |
|   Mixed_6c.branch7x7_1.conv.weight   |   122880   |
|    Mixed_6c.branch7x7_1.bn.weight    |    160     |
|     Mixed_6c.branch7x7_1.bn.bias     |    160     |
|   Mixed_6c.branch7x7_2.conv.weight   |   179200   |
|    Mixed_6c.branch7x7_2.bn.weight    |    160     |
|     Mixed_6c.branch7x7_2.bn.bias     |    160     |
|   Mixed_6c.branch7x7_3.conv.weight   |   215040   |
|    Mixed_6c.branch7x7_3.bn.weight    |    192     |
|     Mixed_6c.branch7x7_3.bn.bias     |    192     |
| Mixed_6c.branch7x7dbl_1.conv.weight  |   122880   |
|  Mixed_6c.branch7x7dbl_1.bn.weight   |    160     |
|   Mixed_6c.branch7x7dbl_1.bn.bias    |    160     |
| Mixed_6c.branch7x7dbl_2.conv.weight  |   179200   |
|  Mixed_6c.branch7x7dbl_2.bn.weight   |    160     |
|   Mixed_6c.branch7x7dbl_2.bn.bias    |    160     |
| Mixed_6c.branch7x7dbl_3.conv.weight  |   179200   |
|  Mixed_6c.branch7x7dbl_3.bn.weight   |    160     |
|   Mixed_6c.branch7x7dbl_3.bn.bias    |    160     |
| Mixed_6c.branch7x7dbl_4.conv.weight  |   179200   |
|  Mixed_6c.branch7x7dbl_4.bn.weight   |    160     |
|   Mixed_6c.branch7x7dbl_4.bn.bias    |    160     |
| Mixed_6c.branch7x7dbl_5.conv.weight  |   215040   |
|  Mixed_6c.branch7x7dbl_5.bn.weight   |    192     |
|   Mixed_6c.branch7x7dbl_5.bn.bias    |    192     |
|   Mixed_6c.branch_pool.conv.weight   |   147456   |
|    Mixed_6c.branch_pool.bn.weight    |    192     |
|     Mixed_6c.branch_pool.bn.bias     |    192     |
|    Mixed_6d.branch1x1.conv.weight    |   147456   |
|     Mixed_6d.branch1x1.bn.weight     |    192     |
|      Mixed_6d.branch1x1.bn.bias      |    192     |
|   Mixed_6d.branch7x7_1.conv.weight   |   122880   |
|    Mixed_6d.branch7x7_1.bn.weight    |    160     |
|     Mixed_6d.branch7x7_1.bn.bias     |    160     |
|   Mixed_6d.branch7x7_2.conv.weight   |   179200   |
|    Mixed_6d.branch7x7_2.bn.weight    |    160     |
|     Mixed_6d.branch7x7_2.bn.bias     |    160     |
|   Mixed_6d.branch7x7_3.conv.weight   |   215040   |
|    Mixed_6d.branch7x7_3.bn.weight    |    192     |
|     Mixed_6d.branch7x7_3.bn.bias     |    192     |
| Mixed_6d.branch7x7dbl_1.conv.weight  |   122880   |
|  Mixed_6d.branch7x7dbl_1.bn.weight   |    160     |
|   Mixed_6d.branch7x7dbl_1.bn.bias    |    160     |
| Mixed_6d.branch7x7dbl_2.conv.weight  |   179200   |
|  Mixed_6d.branch7x7dbl_2.bn.weight   |    160     |
|   Mixed_6d.branch7x7dbl_2.bn.bias    |    160     |
| Mixed_6d.branch7x7dbl_3.conv.weight  |   179200   |
|  Mixed_6d.branch7x7dbl_3.bn.weight   |    160     |
|   Mixed_6d.branch7x7dbl_3.bn.bias    |    160     |
| Mixed_6d.branch7x7dbl_4.conv.weight  |   179200   |
|  Mixed_6d.branch7x7dbl_4.bn.weight   |    160     |
|   Mixed_6d.branch7x7dbl_4.bn.bias    |    160     |
| Mixed_6d.branch7x7dbl_5.conv.weight  |   215040   |
|  Mixed_6d.branch7x7dbl_5.bn.weight   |    192     |
|   Mixed_6d.branch7x7dbl_5.bn.bias    |    192     |
|   Mixed_6d.branch_pool.conv.weight   |   147456   |
|    Mixed_6d.branch_pool.bn.weight    |    192     |
|     Mixed_6d.branch_pool.bn.bias     |    192     |
|    Mixed_6e.branch1x1.conv.weight    |   147456   |
|     Mixed_6e.branch1x1.bn.weight     |    192     |
|      Mixed_6e.branch1x1.bn.bias      |    192     |
|   Mixed_6e.branch7x7_1.conv.weight   |   147456   |
|    Mixed_6e.branch7x7_1.bn.weight    |    192     |
|     Mixed_6e.branch7x7_1.bn.bias     |    192     |
|   Mixed_6e.branch7x7_2.conv.weight   |   258048   |
|    Mixed_6e.branch7x7_2.bn.weight    |    192     |
|     Mixed_6e.branch7x7_2.bn.bias     |    192     |
|   Mixed_6e.branch7x7_3.conv.weight   |   258048   |
|    Mixed_6e.branch7x7_3.bn.weight    |    192     |
|     Mixed_6e.branch7x7_3.bn.bias     |    192     |
| Mixed_6e.branch7x7dbl_1.conv.weight  |   147456   |
|  Mixed_6e.branch7x7dbl_1.bn.weight   |    192     |
|   Mixed_6e.branch7x7dbl_1.bn.bias    |    192     |
| Mixed_6e.branch7x7dbl_2.conv.weight  |   258048   |
|  Mixed_6e.branch7x7dbl_2.bn.weight   |    192     |
|   Mixed_6e.branch7x7dbl_2.bn.bias    |    192     |
| Mixed_6e.branch7x7dbl_3.conv.weight  |   258048   |
|  Mixed_6e.branch7x7dbl_3.bn.weight   |    192     |
|   Mixed_6e.branch7x7dbl_3.bn.bias    |    192     |
| Mixed_6e.branch7x7dbl_4.conv.weight  |   258048   |
|  Mixed_6e.branch7x7dbl_4.bn.weight   |    192     |
|   Mixed_6e.branch7x7dbl_4.bn.bias    |    192     |
| Mixed_6e.branch7x7dbl_5.conv.weight  |   258048   |
|  Mixed_6e.branch7x7dbl_5.bn.weight   |    192     |
|   Mixed_6e.branch7x7dbl_5.bn.bias    |    192     |
|   Mixed_6e.branch_pool.conv.weight   |   147456   |
|    Mixed_6e.branch_pool.bn.weight    |    192     |
|     Mixed_6e.branch_pool.bn.bias     |    192     |
|     AuxLogits.conv0.conv.weight      |   98304    |
|      AuxLogits.conv0.bn.weight       |    128     |
|       AuxLogits.conv0.bn.bias        |    128     |
|     AuxLogits.conv1.conv.weight      |  2457600   |
|      AuxLogits.conv1.bn.weight       |    768     |
|       AuxLogits.conv1.bn.bias        |    768     |
|         AuxLogits.fc.weight          |   150528   |
|          AuxLogits.fc.bias           |    196     |
|   Mixed_7a.branch3x3_1.conv.weight   |   147456   |
|    Mixed_7a.branch3x3_1.bn.weight    |    192     |
|     Mixed_7a.branch3x3_1.bn.bias     |    192     |
|   Mixed_7a.branch3x3_2.conv.weight   |   552960   |
|    Mixed_7a.branch3x3_2.bn.weight    |    320     |
|     Mixed_7a.branch3x3_2.bn.bias     |    320     |
|  Mixed_7a.branch7x7x3_1.conv.weight  |   147456   |
|   Mixed_7a.branch7x7x3_1.bn.weight   |    192     |
|    Mixed_7a.branch7x7x3_1.bn.bias    |    192     |
|  Mixed_7a.branch7x7x3_2.conv.weight  |   258048   |
|   Mixed_7a.branch7x7x3_2.bn.weight   |    192     |
|    Mixed_7a.branch7x7x3_2.bn.bias    |    192     |
|  Mixed_7a.branch7x7x3_3.conv.weight  |   258048   |
|   Mixed_7a.branch7x7x3_3.bn.weight   |    192     |
|    Mixed_7a.branch7x7x3_3.bn.bias    |    192     |
|  Mixed_7a.branch7x7x3_4.conv.weight  |   331776   |
|   Mixed_7a.branch7x7x3_4.bn.weight   |    192     |
|    Mixed_7a.branch7x7x3_4.bn.bias    |    192     |
|    Mixed_7b.branch1x1.conv.weight    |   409600   |
|     Mixed_7b.branch1x1.bn.weight     |    320     |
|      Mixed_7b.branch1x1.bn.bias      |    320     |
|   Mixed_7b.branch3x3_1.conv.weight   |   491520   |
|    Mixed_7b.branch3x3_1.bn.weight    |    384     |
|     Mixed_7b.branch3x3_1.bn.bias     |    384     |
|  Mixed_7b.branch3x3_2a.conv.weight   |   442368   |
|   Mixed_7b.branch3x3_2a.bn.weight    |    384     |
|    Mixed_7b.branch3x3_2a.bn.bias     |    384     |
|  Mixed_7b.branch3x3_2b.conv.weight   |   442368   |
|   Mixed_7b.branch3x3_2b.bn.weight    |    384     |
|    Mixed_7b.branch3x3_2b.bn.bias     |    384     |
| Mixed_7b.branch3x3dbl_1.conv.weight  |   573440   |
|  Mixed_7b.branch3x3dbl_1.bn.weight   |    448     |
|   Mixed_7b.branch3x3dbl_1.bn.bias    |    448     |
| Mixed_7b.branch3x3dbl_2.conv.weight  |  1548288   |
|  Mixed_7b.branch3x3dbl_2.bn.weight   |    384     |
|   Mixed_7b.branch3x3dbl_2.bn.bias    |    384     |
| Mixed_7b.branch3x3dbl_3a.conv.weight |   442368   |
|  Mixed_7b.branch3x3dbl_3a.bn.weight  |    384     |
|   Mixed_7b.branch3x3dbl_3a.bn.bias   |    384     |
| Mixed_7b.branch3x3dbl_3b.conv.weight |   442368   |
|  Mixed_7b.branch3x3dbl_3b.bn.weight  |    384     |
|   Mixed_7b.branch3x3dbl_3b.bn.bias   |    384     |
|   Mixed_7b.branch_pool.conv.weight   |   245760   |
|    Mixed_7b.branch_pool.bn.weight    |    192     |
|     Mixed_7b.branch_pool.bn.bias     |    192     |
|    Mixed_7c.branch1x1.conv.weight    |   655360   |
|     Mixed_7c.branch1x1.bn.weight     |    320     |
|      Mixed_7c.branch1x1.bn.bias      |    320     |
|   Mixed_7c.branch3x3_1.conv.weight   |   786432   |
|    Mixed_7c.branch3x3_1.bn.weight    |    384     |
|     Mixed_7c.branch3x3_1.bn.bias     |    384     |
|  Mixed_7c.branch3x3_2a.conv.weight   |   442368   |
|   Mixed_7c.branch3x3_2a.bn.weight    |    384     |
|    Mixed_7c.branch3x3_2a.bn.bias     |    384     |
|  Mixed_7c.branch3x3_2b.conv.weight   |   442368   |
|   Mixed_7c.branch3x3_2b.bn.weight    |    384     |
|    Mixed_7c.branch3x3_2b.bn.bias     |    384     |
| Mixed_7c.branch3x3dbl_1.conv.weight  |   917504   |
|  Mixed_7c.branch3x3dbl_1.bn.weight   |    448     |
|   Mixed_7c.branch3x3dbl_1.bn.bias    |    448     |
| Mixed_7c.branch3x3dbl_2.conv.weight  |  1548288   |
|  Mixed_7c.branch3x3dbl_2.bn.weight   |    384     |
|   Mixed_7c.branch3x3dbl_2.bn.bias    |    384     |
| Mixed_7c.branch3x3dbl_3a.conv.weight |   442368   |
|  Mixed_7c.branch3x3dbl_3a.bn.weight  |    384     |
|   Mixed_7c.branch3x3dbl_3a.bn.bias   |    384     |
| Mixed_7c.branch3x3dbl_3b.conv.weight |   442368   |
|  Mixed_7c.branch3x3dbl_3b.bn.weight  |    384     |
|   Mixed_7c.branch3x3dbl_3b.bn.bias   |    384     |
|   Mixed_7c.branch_pool.conv.weight   |   393216   |
|    Mixed_7c.branch_pool.bn.weight    |    192     |
|     Mixed_7c.branch_pool.bn.bias     |    192     |
|              fc.weight               |   401408   |
|               fc.bias                |    196     |
+--------------------------------------+------------+
Total Trainable Params: 24895592
24895592